# Homework Assignment 2: QR Factorization

One algorithmic idea in numerical linear algebra is more important ([one of the 10 most important algorithms of the 20th century](https://archive.siam.org/pdf/news/637.pdf)) than all the others: QR factorization. This decomposition plays a crucial role in many problems:
* Computing orthogonal bases in a linear space.
* The preprocessing step for the singular value decomposition (principal component analysis).
* The computation of eigenvectors and eigenvalues (covered in next week's lectures).
* Methods for least squares problems (overdetermined systems of linear equations).

Before tackling the task of implementing this method, let's first introduce the Python library that provides all the necessary linear algebra functionality.

In [ ]:
# All Import Statements Defined Here
# Note: Do not add to this list anywhere.
# ----------------

import math
import numpy as np
import numpy.linalg as la

## Standard Gram-Schmidt Process

Recall that every rectangular $n \times m$ ($n \geq m$) matrix $A$ with linearly indepedent columns can be represented as a product
$$A = QR, \qquad \text{or in matrix form} \qquad \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_m \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} r_{11} & r_{12} & \dots & r_{1m} \\ & r_{22} & \dots & r_{2m}\\ & & \ddots & \vdots \\ & & & r_{mm} \end{bmatrix}$$
where $Q$ is an orthogonal $n \times m$ matrix (i.e. its columns are orthonormal) and $R$ is an upper-triangular $m \times m$ matrix. In this assignment your task will be to implement several methods to achieve this factorization for a given matrix $A$. 

Our first goal is to orthogonalize a system of linearly independent vectors $v_1, \dots, v_m$ (i.e. the columns of some matrix $A$). The standard procedure for this task is the Gram-Schmidt process as covered in the lecture:
\begin{align*}
u_1 &= \frac{v_1}{r_{11}} \\
v_2^\perp &= v_2 - r_{12} u_1, \quad u_2 = \frac{v_2^\perp}{r_{22}} \\
v_3^\perp &= v_3 - r_{13} u_1 - r_{23} u_2, \quad u_3 = \frac{v_2^\perp}{r_{33}} \\
\vdots
\end{align*}
where
\begin{align*}
r_{jj} &= \|v_j - \sum_{i=1}^{j-1} r_{ij} u_i\| &&\quad \text{for }\ 1 \leq j \leq m\ \\
r_{ij} &= \langle v_j, u_i \rangle &&\quad \text{for } 1 \leq i < j \leq m
\end{align*}

## Problem 2: (Classical) Gram-Schmidt Orthogonalization (3 points)

Implement the described Gram-Schmidt algorithm as a function `gram_schmidt_qr` below that takes a rectangular array $A$ and outputs $Q$ and $R$. 

**IMPORTANT**: Do not use any built-in QR functions like `np.linalg.qr` or `scipy.lingalg.qr`. These will give different results anyway as these use a different numerical scheme. 

**HINT**: The following functions for vectors might be useful: `np.linalg.norm` to compute the norm of a 1-D array and `np.inner` to compute the regular dot product between two 1-D arrays.

In [94]:
def gram_schmidt_qr(A):
    """
    Computes the QR factorization of a matrix A with linearly independent columns
    by means of the classical Gram-Schmidt process, where Q is an orthogonal matrix 
    and R is upper-triangular with positive entries on its diagonal. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by m
        R (ndarray): A two-dimensional array of size m by m
        
    """
    
    n, m = A.shape
    Q = np.zeros((n,m))
    R = np.zeros((m,m))
    
    # YOUR CODE HERE
    for j in range(1, m+1):
        suma = 0
        persuma = 0
        # upper triangular matrix R, column by column
        for i in range(1, j):
            R[i-1,j-1] = np.inner(np.hstack(A[:,j-1:j]), np.hstack(Q[:,i-1:i])) #r(ij)
            suma += R[i-1,j-1]*Q[:,i-1:i] #r(ij)*u(i)
        R[j-1,j-1] = np.linalg.norm(A[:,j-1:j]-suma) #r(jj)
        # orthogonal matrix Q, column by column
        if j>1:
            for i in range(1,j):
                persuma += R[i-1, j-1]*Q[:,i-1:i] #v(j) perpendicular 
        Q[:,j-1:j] = (A[:,j-1:j]-persuma) / R[j-1,j-1] #u(j)
            
    return Q, R


In [89]:
# You can use this code cell to play around with your function to make sure
# it does what it is intended to do, i.e. to debug your code. 
print("Original array\n", arr, "\n")

print("Subarray using arr[:,0:1]\n", arr[:,2:3])
a = arr[:,1:2]- 0
print(a)


print("\n next \n")
m=3
for i in range(1, 2):
    print(i)

print("\n next \n")

A = np.array([[2,2], [1,7], [-2, -8]])
n, m = A.shape
Q = np.zeros((n,m))
R = np.zeros((m,m))
for j in range(1, m+1):
        for i in range(1, j):
            aj=np.hstack(A[:,j-1:j])
            qi=np.hstack(Q[:,i-1:i])
            R[i-1,j-1]=np.inner(aj, qi)
            print(R[i-1,j-1])





Original array
 [[2 3 4]
 [4 5 6]
 [6 7 8]] 

Subarray using arr[:,0:1]
 [[4]
 [6]
 [8]]
[[3]
 [5]
 [7]]

 next 

1

 next 

0.0


In [91]:
# Test case I
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this

A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
print(Q, "\n")
print(R)

[[ 0.66666667 -0.66666667]
 [ 0.33333333  0.66666667]
 [-0.66666667 -0.33333333]] 

[[3. 9.]
 [0. 6.]]


Expected output:

    [[ 0.66666667 -0.66666667]
     [ 0.33333333  0.66666667]
     [-0.66666667 -0.33333333]]
     
    [[3. 9.]
     [0. 6.]]

In [92]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = gram_schmidt_qr(B)
print(Q, "\n")
print(R)

[[-0.5  0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0.5]
 [-0.5  0.5 -0.5]] 

[[ 2. -5.  2.]
 [ 0.  3. -2.]
 [ 0.  0.  2.]]


Expected output:

    [[-0.5  0.5  0.5]
     [ 0.5  0.5 -0.5]
     [ 0.5  0.5  0.5]
     [-0.5  0.5 -0.5]]
     
    [[ 2. -5.  2.]
     [ 0.  3. -2.]
     [ 0.  0.  2.]]

In [95]:
# AUTOGRADER
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
Q_exact, R_exact = 1/3*np.array([[2, -2], [1, 2], [-2, -1]]), np.array([[3, 9], [0, 6]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)

B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = gram_schmidt_qr(B)
Q_exact, R_exact = 1/2 * np.array([[-1, 1, 1], [1, 1, -1], [1, 1, 1], [-1, 1, -1]]), np.array([[2, -5, 2], [0, 3, -2], [0, 0, 2]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)


## Householder Triangularization

Another principal method for computing QR factorizations is Householder triangularization, which is numerically more stable than Gram-Schmidt orthogonalization. The Householder algorithm is a process of "orthogonal triangularization", making a matrix triangular by a sequence of orthogonal matrix operations. 

In contrast to the Gram-Schmidt iteration which applies a succession of elementary triangular matrices $R_i$ on the right of $A$ resulting in an orthogonal matrix, the Householder method applies a succession of elementary orthogonal matrices $Q_i$ on the left of $A$ so that the resulting matrix
$$Q_m \dots Q_2 Q_1 A = R$$
is upper-triangular. The product $Q = Q_1^T Q_2^T \dots Q_m^T$ is also orthogonal, and therefore $A = QR$ is our $QR$ factorization of $A$. We can summarize this method as orthogonal triangularization. 

At the heart of the Householder method are orthogonal matrices $Q_i$ to introduce zeroes as indicated below. In these matrices, the symbol $\times$ represents an entry that is not necesssarily zero, and boldfacing them indicates an entry that has just been changed. Blank entries are zero. 

$$\begin{bmatrix}
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times 
\end{bmatrix} \xrightarrow{Q_1} \begin{bmatrix}
\pmb{\times} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} 
\end{bmatrix} \xrightarrow{Q_2} \begin{bmatrix}
\times & \times & \times \\ 
 & \pmb{\times} & \pmb{\times} \\ 
 & \pmb{0} & \pmb{\times} \\ 
 & \pmb{0} & \pmb{\times} \\ 
 & \pmb{0} & \pmb{\times} 
\end{bmatrix} \xrightarrow{Q_3} \begin{bmatrix}
\times & \times & \times \\ 
 & \times & \times \\ 
 &  & \pmb{\times} \\ 
 &  & \pmb{0} \\ 
 &  & \pmb{0} 
\end{bmatrix}$$

In general $Q_k$ operates on rows $k, \dots, n$ and columns $k, \dots, m$. At the beginning of step $k$, there is a block of zeroes in the first $k-1$ columns of these rows. The application of $Q_k$ forms linear combinations of these rows, and the linear combinations of the zero entries remain zero. 

### Householder Reflectors

So how do we construct orthogonal matrices $Q_k$ to introduce zeroes as indicated above? The standard approach is as follows. Each $Q_k$ is chosen to be an orthogonal matrix of the form
$$Q_k = \begin{bmatrix} I & 0 \\ 0 & F \end{bmatrix} \label{eq_Q}\tag{1},$$
where $I$ is the $(k-1) \times (k-1)$ identity matrix and $F$ is an $(n-k+1) \times (n - k + 1)$ orthogonal matrix. The identity matrix will make sure that we don't alter the first $k-1$ columns that already have zeroes introduced in the right replaces, and multiplication by $F$ must introduce zeroes into the $k$th column. The Householder algorithm chooses $F$ to be a particular matrix called a *Householder reflector*. 

Suppose, at the beginning of step $k$, the entries $k, \dots, n$ of the $k$th column are given by a vector $x \in \mathbb{R}^{n-k+1}$. To introduce the correct zeroes into the $k$th column, the Householder reflector $F$ should effect the  following map (recall an orthogonal matrix is length preserving):
$$x = \begin{bmatrix} \times \\ \times \\ \vdots \\ \times \end{bmatrix} \xrightarrow{F} Fx = \begin{bmatrix} \|x\| \\ 0 \\ \vdots \\ 0 \end{bmatrix} = \|x\| e_1.$$
(We shall modify this idea by a $\pm$ sign in a moment). The idea for accomplishing this is indicated in the figure below. 

<img src="https://i.ibb.co/gRhhyzW/householder.png" alt="householder" border="0" width=500></a>

The reflector $F$ will reflect the space $\mathbb{R}^{n-k+1}$ across the hyperplane $H$ orthogonal to the vector $v = \|x\|e_1 - x$. The formula for this reflection can be derived as follows. We know that the vector 
$$Py = \left(I - \frac{v v^T}{v^T v}\right)y = y - v \left(\frac{v^T y}{v^T v}\right) = y - \frac{v}{\|v\|} \left\langle \frac{v}{\|v\|}, y \right\rangle = y - \langle u, y \rangle u$$
is the orthogonal projection of $y$ onto the space $H$. To reflect $y$ across $H$, we must not stop at this point; we must go exactly twice as far in the same direction. The reflection $Fy$ should therefore be
$$Fy = \left(I - 2\frac{v v^T}{v^T v}\right)y = y - 2v \left(\frac{v^T y}{v^T v}\right).$$
Hence the matrix $F$ is 
$$F = I - 2 \frac{v v^T}{v^T v}, \quad \text{with} \quad v = \|x\| e_1 - x.$$

### The Better of Two Reflectors

In the above figure we have simplified matters, for in fact, there are two Householder reflections that will introduce the zeroes needed. In the figure below two alternatives are represented by reflections across two different hyperplanes, $H^+$ and $H^-$. 

<img src="https://i.ibb.co/dDNSbS0/householder2.png" alt="householder2" border="0" width=500></a>

Mathematically either choice of sign is satisfactory. However, numerical stability (insensitivity to rounding errors) dictates that one choice should be taken rather than the other. For numerical stability, it is desirable to reflect $x$ to the vector $z \|x\| e_1$ that is not too close to $x$ itself. To achieve this, we can choose $z = -\text{sign}(x_1)$, where $x_1$ denotes the first component of $x$, so that the reflection vector becomes $v = -\text{sign}(x_1) \|x\| e_1 - x$, or upon clearing the factors $-1$,
$$v = \text{sign}(x_1) \|x\| e_1 + x.$$
To make a complete prescription, we may arbitrarily impose the convention that $\text{sign}(x_1) = 1$ if $x_1 = 0$. 

To see why the choice of sign makes a difference for stability, suppose that the angle between $H^+$ and the $e_1$-axis is very small. Then the vector $v = \|x\|e_1 - x$ is much smaller than $x$ or $\|x\|e_1$. Thus the calculation of $v$ represents a subtraction of nearby quantities and will tend to suffer from cancellation errors. By picking the opposite sign, we avoid such effects by ensuring that $\|v\|$ is never smaller than $\|x\|$. 

**Note**: In this QR factorization the rectangular $n \times m$ ($n \geq m$) matrix $A$ with linearly indepedent columns will be represented as a product
$$A = QR, \qquad \text{or in matrix form} \qquad \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_n \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} r_{11} & r_{12} & \dots & r_{1m} \\ & r_{22} & \dots & r_{2m}\\ & & \ddots & \vdots \\ & & & r_{mm} \\ & & & \end{bmatrix}$$
where $Q$ is an orthogonal $n \times n$ matrix and $R$ is an upper-triangular $n \times m$ matrix. So when $A$ is not square, $R$ will contain extra rows of zeroes at the bottom. Moreover, we also allow for negative entries along the diagonal (as opposed to the QR factorization we obtain by applying the Gram-Schmidt process). 

## Problem 3: Householder QR Factorization (5 = 2 + 3 points)

Implement both the functions `householder_reflector` and `householder_qr` below. 

**Hint**: The function [`np.outer`](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) might be useful to compute $v v^T$ when we represent $v$ as a 1 dimensional array. The function [`np.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) is useful to compute the norm of a vector. The function [`np.inner`](https://numpy.org/doc/stable/reference/generated/numpy.inner.html) is useful to compute the dot product of two vectors. 

You can multiply two 2 dimensional arrays (i.e. matrices) $A$ and $B$ by `A @ B` or by `np.matmul(A,B)`. You can compute the inverse of a matrix by [`np.lingalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html).

In [450]:
def sign(x):
    """
    To be used in householder_qr below in the computation of
        v = sign(x) * norm(x) * e_1 + x
    """
    if(x >= 0):
        return(1)
    else:
        return(-1)

def householder_reflector(v, k, n):
    """
    Returns the orthogonal n by n Householder reflector Q_k as described by equation (1) above,
    corresponding to the hyperplane determined by the vector v.
    
    Parameters
    ----------
        v (ndarray): A 1 dimensional array of length n-k+1. 
        k (int): Denotes the required Householder reflector
                 BEWARE THAT PYTHON INDEXING STARTS AT ZERO,
                 THIS MIGHT BE CONFUSING WITH THE WAY Q_K IS
                 NUMBERED.
        n (int): Number of rows and columns of Q_k
    
    Returns
    -------
        Q_k (ndarray): A 2 dimensional n by n array. 
    
    """
    
    # YOUR CODE HERE
    I=np.eye(k-1)
    n_rows=np.ones((k-1,1), dtype=int)
    n_col=np.zeros((1,n-k+1), dtype=int)
    I_plus = np.concatenate([I, n_rows.dot(n_col)], axis=1)

    vt=np.transpose([v])
    a=v*vt
    F=np.eye(n-k+1)-2*a/np.linalg.norm(a)
    n_rows=np.ones((n-k+1,1), dtype=int)
    n_col=np.zeros((1,k-1), dtype=int)
    F_plus = np.concatenate([n_rows.dot(n_col), F], axis=1)
    
    Q_k=I_plus
    for i in range(n-k+1):
        Q_k=np.r_[Q_k, [F_plus[i]]]
    return(Q_k)

    
def householder_qr(A):
    """
    Computes a QR factorization of a matrix A by means of Householder
    reflectors, where Q is an orthogonal matrix and R is upper-triangular. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by n
        R (ndarray): A two-dimensional array of size n by m
    
    """
    
    # YOUR CODE HERE
    Ri = A
    n, m = A.shape
    Q = np.eye((n))
    for i in range(1,n): 
        xi = Ri[:,i-1:i]
        e1=np.r_[[[1]],np.zeros((n-i,1), dtype=float)]
        
        if i>1:
            for j in range(i-1):
                xi = np.delete(xi, 0, axis=0)
        vi = np.linalg.norm(xi) * e1 * sign(xi[0,0]) + xi
        vi = np.hstack(vi)
        Qi = householder_reflector(vi,i,n)
        Ri=Qi @ Ri
        Q = Q @ Qi.T
    R = Ri
    return(Q, R)
    

In [447]:
# You can use this code cell to play around with your function to make sure
# it does what it is intended to do, i.e. to debug your code. 
#A = np.array([[-1,-1,1], [1,3,3], [-1, -1,5]])
#A = np.array([[1, -1, 4], [1, 4, -2], [1, 4, 2], [1, -1, 0]])

A= np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])

n, m = A.shape
Ri = A
Q = np.eye((n))
for i in range(1,n): 
    xi = Ri[:,i-1:i]
    e1=np.r_[[[1]],np.zeros((n-i,1))]
    if i>1:
        for j in range(i-1):
            xi = np.delete(xi, 0, axis=0)
    #vi = xi - np.linalg.norm(xi) * e1 
    vi = np.linalg.norm(xi) * e1 * sign(xi[0,0]) + xi
    vi = np.hstack(vi)
    Qi = householder_reflector(vi,i,n)
    Ri=Qi @ Ri
    Q = Q @ Qi
    print("\n\n i=", i, "\n xi \n", xi, "\n\n vi \n", vi, "\n\n Qi \n", Qi, "\n\n Ri \n", Ri, "\n\n Q \n", Q, "\n\n")
R = Ri
#print(R, "\n\n Q \n", Q)
    


# A = np.array([[2,2], [1,7], [-2, -8]])
# Q, R = householder_qr(A)
# Q_exact, R_exact = 1/3*np.array([[-2, 2, 1], [-1, -2, 2], [2, 1, 2]]), np.array([[-3, -9], [0, -6], [0, 0]])
# print(Q, "\n \n" ,R, "\n \n")
# print(Q_exact, "\n \n" ,R_exact, "\n \n")
    
    

# k=2
# v = vi
# I=np.eye(k-1)
# n_rows=np.ones((k-1,1), dtype=int)
# n_col=np.zeros((1,n-k+1), dtype=int)
# I_plus = np.concatenate([I, n_rows.dot(n_col)], axis=1)

# vt=np.transpose([v])
# a=v*vt
# print("check")
# print(np.eye(n-k+1),"\n\n", 2*a)
# F=np.eye(n-k+1)-2*a/np.linalg.norm(a)
# n_rows=np.ones((n-k+1,1), dtype=int)
# n_col=np.zeros((1,k-1), dtype=int)
# F_plus = np.concatenate([n_rows.dot(n_col), F], axis=1)
    
# Q_k=I_plus
# for i in range(n-k+1):
#     Q_k=np.r_[Q_k, [F_plus[i]]]
# return(Q_k)




 i= 1 
 xi 
 [[-1]
 [ 1]
 [ 1]
 [-1]] 

 vi 
 [-3.  1.  1. -1.] 

 Qi 
 [[-0.5         0.5         0.5        -0.5       ]
 [ 0.5         0.83333333 -0.16666667  0.16666667]
 [ 0.5        -0.16666667  0.83333333  0.16666667]
 [-0.5         0.16666667  0.16666667  0.83333333]] 

 Ri 
 [[ 2. -5.  2.]
 [ 0.  2. -2.]
 [ 0.  2.  0.]
 [-0.  1. -2.]] 

 Q 
 [[-0.5         0.5         0.5        -0.5       ]
 [ 0.5         0.83333333 -0.16666667  0.16666667]
 [ 0.5        -0.16666667  0.83333333  0.16666667]
 [-0.5         0.16666667  0.16666667  0.83333333]] 




 i= 2 
 xi 
 [[2.]
 [2.]
 [1.]] 

 vi 
 [5. 2. 1.] 

 Qi 
 [[ 1.          0.          0.          0.        ]
 [ 0.         -0.66666667 -0.66666667 -0.33333333]
 [ 0.         -0.66666667  0.73333333 -0.13333333]
 [ 0.         -0.33333333 -0.13333333  0.93333333]] 

 Ri 
 [[ 2.  -5.   2. ]
 [-0.  -3.   2. ]
 [ 0.  -0.   1.6]
 [-0.   0.  -1.2]] 

 Q 
 [[-0.5 -0.5  0.1 -0.7]
 [ 0.5 -0.5 -0.7 -0.1]
 [ 0.5 -0.5  0.7  0.1]
 [-0.5 -0.5 -0

In [451]:
# Test case for householder_reflector
v1 = np.array([5, 1, -2])
Q_1 = householder_reflector(v1, 1, 3)
print(Q_1, "\n")

v2 = np.array([10.8, -3.6])
Q_2 = householder_reflector(v2, 2, 3)
print(Q_2)

[[-0.66666667 -0.33333333  0.66666667]
 [-0.33333333  0.93333333  0.13333333]
 [ 0.66666667  0.13333333  0.73333333]] 

[[ 1.   0.   0. ]
 [ 0.  -0.8  0.6]
 [ 0.   0.6  0.8]]


Expected output:

    [[-0.66666667 -0.33333333  0.66666667]
     [-0.33333333  0.93333333  0.13333333]
     [ 0.66666667  0.13333333  0.73333333]]
     
    [[ 1.   0.   0. ]
     [ 0.  -0.8  0.6]
     [ 0.   0.6  0.8]]

In [452]:
# Test case for householder_qr
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this

A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = householder_qr(A)
print(Q, "\n")
print(R)

[[-0.66666667  0.66666667  0.33333333]
 [-0.33333333 -0.66666667  0.66666667]
 [ 0.66666667  0.33333333  0.66666667]] 

[[-3. -9.]
 [-0. -6.]
 [-0.  0.]]


Expected output:

    [[-0.66666667  0.66666667  0.33333333]
     [-0.33333333 -0.66666667  0.66666667]
     [ 0.66666667  0.33333333  0.66666667]]
     
    [[-3. -9.]
     [-0. -6.]
     [-0.  0.]]

In [453]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = householder_qr(B)
print(Q, "\n")
print(R)

[[-0.5 -0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]
 [-0.5 -0.5  0.5  0.5]] 

[[ 2. -5.  2.]
 [-0. -3.  2.]
 [-0.  0. -2.]
 [-0.  0.  0.]]


Expected output:

    [[-0.5 -0.5 -0.5 -0.5]
     [ 0.5 -0.5  0.5 -0.5]
     [ 0.5 -0.5 -0.5  0.5]
     [-0.5 -0.5  0.5  0.5]]
     
    [[ 2. -5.  2.]
     [ 0. -3.  2.]
     [ 0.  0. -2.]
     [ 0. -0.  0.]]

In [176]:
# AUTOGRADER for householder_reflector
v1 = np.array([5, 1, -2])
Q_1 = householder_reflector(v1, 1, 3)
assert np.allclose(Q_1, 1/15 * np.array([[-10, -5, 10], [-5, 14, 2], [10, 2, 11]]))

v2 = np.array([10.8, -3.6])
Q_2 = householder_reflector(v2, 2, 3)
assert np.allclose(Q_2, 1/5 * np.array([[5, 0, 0], [0, -4, 3], [0, 3, 4]]))

In [454]:
# AUTOGRADER for householder_qr
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = householder_qr(A)
Q_exact, R_exact = 1/3*np.array([[-2, 2, 1], [-1, -2, 2], [2, 1, 2]]), np.array([[-3, -9], [0, -6], [0, 0]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)

B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = householder_qr(B)

Q_exact = 1/2 * np.array([[-1, -1, -1, -1], [1, -1, 1, -1], [1, -1, -1, 1], [-1, -1, 1, 1]]) 
R_exact = np.array([[2, -5, 2], [0, -3, 2], [0, 0, -2], [0, 0, 0]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)


## Optional: Modified Gram-Schmidt Algorithm and Triangular Orthogonalization
### Feel free to skip.

The classical Gram-Schmidt algorithm is numerically unstable, which means that when implemented on a computer, round-off errors can cause the output vectors to be significantly non-orthogonal. This instability can be improved with a small adjustment to the algorithm. In the literature this remedy is called the modified Gram-Schmidt method. Instead of doing
$$v_k^\perp = v_k - \langle v_k, u_1 \rangle u_1 - \dots - \langle v_k, u_{k-1}\rangle u_{k-1}$$
we do the orthogonalization step-by-step. The modified algorithm calculates $u_j$ by evaluating the following formulas in order:
\begin{align*}
v_j^{(1)} &= v_j \\
v_j^{(2)} &= v_j^{(1)} - \langle v_j^{(1)}, u_1 \rangle u_1 \\
v_j^{(3)} &= v_j^{(2)} - \langle v_j^{(2)}, u_2 \rangle u_2 \\
v_j^{(4)} &= v_j^{(3)} - \langle v_j^{(3)}, u_3 \rangle u_3 \\
& ~~ \vdots \\
u_j = v_j^{(j)} &= v_j^{(j-1)} - \langle v_j^{(j-1)}, u_{j-1} \rangle u_{j-1}
\end{align*}

Once a new $u_i$ is known, we can overwrite each $v_j^{(i)}$ to $v_j^{(i+1)}$ for each $j>i$. Each step of the modified Gram-Schmidt algorithm can be interpreted as a right-multiplication by a square upper-triangular matrix. For example, beginning with $A$, the first iteration multiplies the first column $v_1$ by $\frac{1}{r_{11}} = \frac{1}{\|v_1\|}$ and then subtracts $r_{1j} = \langle v_j^{(1)}, u_1 \rangle = \frac{\langle v_j, v_1 \rangle}{\|v_1\|}$ times the result from each of the remaining columns $v_j$. This is equivalent to right-multiplication by a matrix $R_1$:

$$AR_1 = \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} \frac{1}{r_{11}} & \frac{-r_{12}}{r_{11}} & \frac{-r_{13}}{r_{11}} & \dots \\ & 1 &  & \\ & & 1 &  \\ & & & \ddots \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & v_2^{(2)} & \dots & v_m^{(2)} \\ \mid & \mid & & \mid \end{bmatrix}.$$

Empty entries signify zeroes. In general, in step $i$ the algorithm subtracts $r_{ij}/r_{ii}$ times column $i$ of the current $A$ from columns $j > i$ and replaces column $i$ by $1/r_{ii}$ times itself. This corresponds to multiplication by an upper-triangular matrix $R_i$:
$$R_2 = \begin{bmatrix} 1 & & & \\ & \frac{1}{r_{22}} & \frac{-r_{23}}{r_{22}} & \dots\\ & & 1 &  \\ & & & \ddots \end{bmatrix}, \quad R_3 = \begin{bmatrix} 1 & & & \\ & 1 & & \\ & & \frac{1}{r_{33}} & \dots \\ & & & \ddots \end{bmatrix}, \quad \dots$$
Recall that
\begin{align*}
r_{ii} &= \| v_i^{(i)} \|,  &&\text{for } 1 \leq i \leq m;\\
r_{ij} &= \langle u_i, v_j^{(i)} \rangle = \frac{\langle v_i^{(i)}, v_j^{(i)}\rangle}{\|v_i^{(i)}\|},  &&\text{for } 1 \leq i < j \leq m.
\end{align*}

After $i$ multiplications we have
$$A R_1 R_2 \dots R_i = \begin{bmatrix}\mid & \mid & & \mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_i & v_{i+1}^{(i+1)} & \dots & v_{m}^{(i+1)} \\ \mid & \mid & & \mid & \mid & & \mid \end{bmatrix}$$

At the end of the iteration we have
$$A R_1 R_2 \dots R_n = Q,$$
and equivalently $A = QR = Q (R_1 R_2 \dots R_n)^{-1}.$ Hence the name triangular orthogonalization. 